In [1]:
# @title Environment Setup
import os
import sys
if 'MPLBACKEND' in os.environ:
    del os.environ['MPLBACKEND']
    print("MPLBACKEND environment variable cleared.")

# 2. Clone the repository
!rm -rf FontDiffusion
!git clone https://github.com/dzungphieuluuky/FontDiffusion.git

MPLBACKEND environment variable cleared.
Cloning into 'FontDiffusion'...
remote: Enumerating objects: 20331, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 20331 (delta 53), reused 53 (delta 24), pack-reused 20237 (from 1)
Receiving objects: 100% (20331/20331), 277.46 MiB | 39.21 MiB/s, done.
Resolving deltas: 100% (785/785), done.
Updating files: 100% (137/137), done.


In [2]:
import os
import sys
from IPython import get_ipython
from typing import Optional

def configure_environment_paths():
    try:
        if "google.colab" in str(get_ipython()):
            print("✅ Environment: Google Colab")
            base_data_path = "/content/"
            base_output_path = "/content/"
            environment_name = "colab"
        elif os.environ.get("KAGGLE_KERNEL_RUN_TYPE"):
            print("✅ Environment: Kaggle")
            base_data_path = "/kaggle/input/"
            base_output_path = "/kaggle/working/"
            environment_name = "kaggle"
        else:
            print("⚠️ Environment: Local/Unknown")
            base_data_path = "./data/"
            base_output_path = "./output/"
            environment_name = "local"
    except NameError:
        print("⚠️ Non-interactive session. Using local paths.")
        base_data_path = "./data/"
        base_output_path = "./output/"
        environment_name = "local"
    os.makedirs(base_output_path, exist_ok=True)
    print(f"📂 Data Path: {base_data_path}")
    print(f"📦 Output Path: {base_output_path}")
    return base_data_path, base_output_path, environment_name

def load_secret(key_name: str) -> Optional[str]:
    env = ENV_NAME
    secret_value = None
    print(f"Attempting to load secret '{key_name}' from '{env}' environment...")
    try:
        if env == "colab":
            from google.colab import userdata
            secret_value = userdata.get(key_name)
        elif env == "kaggle":
            from kaggle_secrets import UserSecretsClient
            user_secrets = UserSecretsClient()
            secret_value = user_secrets.get_secret(key_name)
        else:
            secret_value = os.getenv(key_name)
        if not secret_value:
            print(f"⚠️ Secret '{key_name}' not found in the {env} environment.")
            return None
        print(f"✅ Successfully loaded secret '{key_name}'.")
        return secret_value
    except Exception as e:
        print(f"❌ An error occurred while loading secret '{key_name}': {e}")
        return None

def print_system_info():
    print("\n🔧 System Information")
    print(f"Python version: {sys.version.split()[0]}")
    try:
        import torch
        print(f"PyTorch version: {torch.__version__}")
        if torch.cuda.is_available():
            print(f"CUDA version: {torch.version.cuda}")
            print(f"GPU count: {torch.cuda.device_count()}")
            for i in range(torch.cuda.device_count()):
                print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
        else:
            print("CUDA not available")
    except ImportError:
        print("PyTorch not installed")
    finally:
      !nvidia-smi

INPUT_PATH, OUTPUT_PATH, ENV_NAME = configure_environment_paths()
is_kaggle = ("kaggle" in ENV_NAME)
is_colab = not is_kaggle
print_system_info()

os.environ["WANDB_API_KEY"] = load_secret('WANDB_API_KEY')
os.environ["HF_TOKEN"] = load_secret('HF_TOKEN')

# Now, these libraries will log in automatically
import wandb
import huggingface_hub

wandb.login()  # Authenticates silently
huggingface_hub.login(token=os.environ["HF_TOKEN"]) 
# Or just rely on HF_TOKEN being in env
HF_USERNAME = "dzungpham"

✅ Environment: Kaggle
📂 Data Path: /kaggle/input/
📦 Output Path: /kaggle/working/

🔧 System Information
Python version: 3.11.13
PyTorch version: 2.6.0+cu124
CUDA version: 12.4
GPU count: 2
  GPU 0: Tesla T4
  GPU 1: Tesla T4
Wed Dec 31 19:04:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A  

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [3]:
!uv pip install --upgrade pip
# 3. Install PyTorch 1.13
%cd {OUTPUT_PATH}
# Force reinstall torch 1.13 to match the model's training environment
# !uv pip uninstall torch torchvision
# !uv pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!uv pip install torch torchvision
# 4. Install other dependencies
print("\n⬇️ Installing Dependencies (Manually fixed)...")
# Install xformers compatible with Torch 1.13
!uv pip install xformers==0.0.16 -q

# Install original dependencies
!uv pip install transformers==4.33.1 accelerate==0.23.0 diffusers==0.22.0
!uv pip install gradio==4.8.0 pyyaml pygame opencv-python info-nce-pytorch kornia
# -----------------------------------------------------------------
!uv pip install lpips scikit-image pytorch-fid
!sudo apt-get update && sudo apt-get install dos2unix
!uv pip install gdown
!uv pip install wandb
!uv pip install --upgrade pyarrow datasets
print("\n✅ Environment setup complete. You can now proceed to Block 2 (Inference).")

Using Python 3.11.13 environment at: /usr
Resolved 1 package in 44ms                                           
Audited 1 package in 0.23ms
/kaggle/working
Using Python 3.11.13 environment at: /usr
Audited 2 packages in 100ms

⬇️ Installing Dependencies (Manually fixed)...
  × Failed to build `xformers==0.0.16`
  ├─▶ The build backend returned an error
  ╰─▶ Call to `setuptools.build_meta:__legacy__.build_wheel` failed (exit
      status: 1)

      [stderr]
      Error in sitecustomize; set PYTHONVERBOSE for traceback:
      ModuleNotFoundError: No module named 'wrapt'
      Traceback (most recent call last):
        File "<string>", line 14, in <module>
        File
      "/root/.cache/uv/builds-v0/.tmpPY7wUW/lib/python3.11/site-packages/setuptools/build_meta.py",
      line 331, in get_requires_for_build_wheel
          return self._get_build_requires(config_settings, requirements=[])
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File
      "/roo

In [4]:
# KAGGLE CELL #1: Download checkpoint

import os
import sys
from pathlib import Path
os.chdir(OUTPUT_PATH)
# Download from Hub
if not os.path.exists("ckpt") or not list(Path("ckpt").glob("*.safetensors")):
    print("📥 Downloading checkpoint from Hugging Face Hub...\n")
    from huggingface_hub import snapshot_download
    snapshot_download(
        repo_id="dzungpham/font-diffusion-weights",
        local_dir="ckpt",
        allow_patterns="*.safetensors",
        force_download=False
    )
    print("\n✅ Download complete!")
else:
    print("✅ Checkpoint already downloaded")
# Verify
print("\n📂 Files in ckpt/:")
for file in os.listdir("ckpt"):
    if file.endswith(".safetensors"):
        size = os.path.getsize(f"ckpt/{file}") / (1024**2)
        print(f"  ✓ {file} ({size:.2f} MB)")

📥 Downloading checkpoint from Hugging Face Hub...



Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]


✅ Download complete!

📂 Files in ckpt/:
  ✓ style_encoder.safetensors (78.58 MB)
  ✓ content_encoder.safetensors (4.54 MB)
  ✓ unet.safetensors (300.34 MB)


In [5]:
# @title Unzipping all archived files
import os
import glob
from zipfile import ZipFile

zip_file_paths = glob.glob(os.path.join(INPUT_PATH, '*.zip'))

if not zip_file_paths:
    print(f'No .zip files found in {INPUT_PATH}.')
else:
    for zip_file_path in zip_file_paths:
        if os.path.exists(zip_file_path):
            print(f'Unzipping {zip_file_path}...')
            !unzip -o {zip_file_path} -d ./
            print(f'Unzipping of {zip_file_path} complete.')
        else:
            print(f'Error: The file {zip_file_path} was not found (post-glob check).')

No .zip files found in /kaggle/input/.


In [6]:
import pandas as pd
import os
def convert_csv_to_chars_txt(input_csv_path: str, output_txt_path: str, column_name: str = 'word'):
    """
    Reads a CSV file, extracts text from a specified column, and writes each character
    to a new line in a plain text file.
    Args:
        input_csv_path (str): The full path to the input CSV file.
        output_txt_path (str): The full path for the output text file.
        column_name (str): The name of the column in the CSV file containing the text.
    """
    if not os.path.exists(input_csv_path):
        print(f"Error: Input CSV file not found at '{input_csv_path}'. Please ensure the file is uploaded.")
        return
    try:
        df = pd.read_csv(input_csv_path)
    except Exception as e:
        print(f"Error reading CSV file '{input_csv_path}': {e}")
        return
    if column_name not in df.columns:
        print(f"Error: Column '{column_name}' not found in the CSV file '{input_csv_path}'.")
        return
    all_characters = []
    for item in df[column_name].astype(str).dropna().tolist():
        for char in item:
            all_characters.append(char)
    os.makedirs(os.path.dirname(output_txt_path), exist_ok=True)
    with open(output_txt_path, "w", encoding="utf-8") as f:
        f.write("\n".join(all_characters))
    print(f"Successfully converted '{input_csv_path}' to '{output_txt_path}', with one character per line.")
print("\n--- Demonstrating function with a dummy CSV file ---")
dummy_csv_path = os.path.join(OUTPUT_PATH, "dummy_data.csv")
dummy_output_txt_path = os.path.join(OUTPUT_PATH, "dummy_chars.txt")
dummy_data = {'word': ['hello', 'world', 'python']}
pd.DataFrame(dummy_data).to_csv(dummy_csv_path, index=False)
print(f"Created a dummy CSV file at: {dummy_csv_path}")
convert_csv_to_chars_txt(dummy_csv_path, dummy_output_txt_path)


--- Demonstrating function with a dummy CSV file ---
Created a dummy CSV file at: /kaggle/working/dummy_data.csv
Successfully converted '/kaggle/working/dummy_data.csv' to '/kaggle/working/dummy_chars.txt', with one character per line.


In [7]:
!ls -larth {OUTPUT_PATH}/ckpt

total 384M
drwxr-xr-x 3 root root 4.0K Dec 31 19:04 .cache
-rw-r--r-- 1 root root 4.6M Dec 31 19:04 content_encoder.safetensors
-rw-r--r-- 1 root root  79M Dec 31 19:04 style_encoder.safetensors
-rw-r--r-- 1 root root 301M Dec 31 19:04 unet.safetensors
drwxr-xr-x 3 root root 4.0K Dec 31 19:04 .
drwxr-xr-x 5 root root 4.0K Dec 31 19:04 ..


In [8]:
%cd {OUTPUT_PATH}
# ==========================================
# EXPORT / DOWNLOAD DATASET COMMANDS
# ==========================================

# Train Split
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/train" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "train" \
  --token HF_TOKEN
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/train_original" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "train_original" \
  --token HF_TOKEN
# Validation: Unseen Both
!python FontDiffusion/export_hf_dataset_to_disk.py \
  --output_dir "my_dataset/val" \
  --repo_id {HF_USERNAME}/font-diffusion-generated-data \
  --split "val" \
  --token HF_TOKEN

/kaggle/working

EXPORTING DATASET TO DISK

📥 Loading dataset from Hub...
   Repository: dzungpham/font-diffusion-generated-data
   Split: train
README.md: 100%|███████████████████████████████| 733/733 [00:00<00:00, 4.26MB/s]
data/train_original-00000-of-00001.parqu(…): 100%|█| 18.5M/18.5M [00:00<00:00, 2
data/train-00000-of-00001.parquet: 100%|███| 7.70M/7.70M [00:01<00:00, 4.12MB/s]
data/val-00000-of-00001.parquet: 100%|███████| 563k/563k [00:00<00:00, 1.04MB/s]
Generating train_original split: 100%|█| 2175/2175 [00:00<00:00, 36325.67 exampl
Generating val split: 100%|███████████| 57/57 [00:00<00:00, 15255.91 examples/s]
✓ Loaded dataset with 912 samples from Hub

Exporting images from dataset...

🎨 Exporting images...
Exporting: 100%|█████████████████████████████| 912/912 [00:02<00:00, 421.70it/s]
✓ Exported 76 content images
✓ Exported 912 target images

💾 Saving results_checkpoint.json...
  ✓ Saved results_checkpoint.json (912 generations)

📊 Metadata Statistics:
  Total generatio

In [11]:
if is_colab:
  !uv pip install --upgrade "huggingface-hub>=0.34.0,<1.0"
else:
  !uv pip install --upgrade "huggingface-hub==0.25.2" "protobuf<5.0.0" "numpy<2.0.0"
%cd {OUTPUT_PATH}
!python FontDiffusion/sample_batch.py \
    --characters "FontDiffusion/NomTuTao/Ds_10k_ChuNom_TuTao.txt" \
    --style_images "FontDiffusion/styles_images" \
    --ckpt_dir "ckpt/" \
    --ttf_path "FontDiffusion/fonts" \
    --output_dir "my_dataset/train_original" \
    --num_inference_steps 20 \
    --guidance_scale 7.5 \
    --start_line 1 \
    --end_line 100 \
    --batch_size 24 \
    --save_interval 1 \
    --channels_last \
    --seed 42 \
    --compile \
    --enable_xformers

Using Python 3.11.13 environment at: /usr
Resolved 14 packages in 78ms                                         
Prepared 2 packages in 91ms                                              
Uninstalled 2 packages in 35ms
Installed 2 packages in 25ms                                
 - numpy==2.4.0
 + numpy==1.26.4
 - protobuf==6.33.0
 + protobuf==4.25.8
/kaggle/working
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
2025-12-31 19:08:05.168759: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory

In [12]:
!python FontDiffusion/create_validation_split.py \
  --data_root my_dataset \
  --val_ratio 0.2 \
  --seed 42


FONTDIFFUSION SPLIT CREATOR (SIMPLIFIED)
✓ Using source directory: my_dataset/train_original

CREATING DATA SPLITS

ANALYZING TRAINING DATA

🔍 Scanning target images...
Styles: 100%|██████████████████████████████████████████████████| 15/15 [00:00<00:00, 1414.22style/s]

✓ Initial scan:
  Styles: 15
  Characters: 141
  Valid (char, style) pairs: 2115

🔍 Auto-detecting font parameter from existing files...
  📂 Found 15 fonts in fonts/ directory
  ✓ Detected font parameter: 'HAN NOM A'
    Verified with: U+34E4_㓤_20242aa0.png

🔍 Validating content images (using font='HAN NOM A')...
Checking content: 100%|██████████████████████████████████████| 141/141 [00:00<00:00, 84934.20char/s]

⚠️  WARNING: 46/141 characters missing content images:
  Examples: ['𠄩', '𠑬', '𠤆', '𠦳', '𠳒', '𠴝', '𠺥', '𡊰', '𡗉', '𡥵']...
  These characters will be excluded from splits

✅ After validation:
  Characters with content images: 95
  Valid (char, style) pairs: 1425
  Content images found: 95

📊 Character distributi

In [13]:
!uv pip install --upgrade pyarrow datasets

Using Python 3.11.13 environment at: /usr
Resolved 39 packages in 158ms                                        
Prepared 3 packages in 1ms                                               
Uninstalled 3 packages in 24ms
Installed 3 packages in 314ms                               
 - fsspec==2025.12.0
 + fsspec==2025.10.0
 - huggingface-hub==0.25.2
 + huggingface-hub==1.2.3
 - numpy==1.26.4
 + numpy==2.4.0


In [ ]:
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/train_original" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "train_original" \
  --private \
  --token "{HF_TOKEN}"

# Train Split
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/train" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "train" \
  --private \
  --token "{HF_TOKEN}"

# Train Split
!python FontDiffusion/create_hf_dataset.py \
  --data_dir "my_dataset/val" \
  --repo_id "{HF_USERNAME}/font-diffusion-generated-data" \
  --split "val" \
  --private \
  --token "{HF_TOKEN}"


In [ ]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# TRAINING PHASE 1
if is_colab:
  !uv pip install --upgrade "huggingface-hub>=0.34.0,<1.0"
else:
  !uv pip install --upgrade "huggingface-hub==0.25.2"
import wandb

# Load your Weights & Biases API key from a secure store
wandb_key = load_secret("WANDB_API_KEY")
wandb.login(key=wandb_key)

# Run the training script with the corrected flag syntax
!accelerate launch FontDiffusion/my_train.py \
    --seed=123 \
    --experience_name=FontDiffuser_training_phase_1 \
    --data_root=my_dataset \
    --output_dir=outputs/FontDiffuser \
    --report_to=wandb \
    --resolution=96 \
    --style_image_size=96 \
    --content_image_size=96 \
    --content_encoder_downsample_size=3 \
    --channel_attn=True \
    --content_start_channel=64 \
    --style_start_channel=64 \
    --train_batch_size=8 \
    --perceptual_coefficient=0.03 \
    --offset_coefficient=0.7 \
    --max_train_steps=200 \
    --ckpt_interval=100 \
    --val_interval=200 \
    --gradient_accumulation_steps=1 \
    --log_interval=50 \
    --learning_rate=1e-4 \
    --lr_scheduler=linear \
    --lr_warmup_steps=10000 \
    --drop_prob=0.1 \
    --mixed_precision=no

In [ ]:
!ls -lr outputs/FontDiffuser

In [ ]:
# TRAINING PHASE 2
!wandb login
!python FontDiffusion/my_train.py \
    --seed=123 \
    --experience_name="FontDiffuser_training_phase_2" \
    --data_root="my_dataset" \
    --output_dir="outputs/FontDiffuser" \
    --report_to="wandb" \
    --phase_2 \
    --phase_1_ckpt_dir="outputs/FontDiffuser/global_step_2000" \
    --scr_ckpt_path="ckpt/scr_210000.pth" \
    --sc_coefficient=0.05 \
    --num_neg=13 \
    --resolution=96 \
    --style_image_size=96 \
    --content_image_size=96 \
    --content_encoder_downsample_size=3 \
    --channel_attn=True \
    --content_start_channel=64 \
    --style_start_channel=64 \
    --train_batch_size=8 \
    --perceptual_coefficient=0.03 \
    --offset_coefficient=0.4 \
    --max_train_steps=100 \
    --ckpt_interval=50 \
    --gradient_accumulation_steps=2 \
    --log_interval=50 \
    --learning_rate=1e-5 \
    --lr_scheduler="constant" \
    --lr_warmup_steps=1000 \
    --drop_prob=0.1 \
    --mixed_precision="no"


In [ ]:
!python FontDiffusion/pth2safetensors.py \
    --weights_dir "ckpt" \
    --repo_id "dzungpham/font-diffusion-weights" \
    --token "{HF_TOKEN}"

In [ ]:
import os
import zipfile
from pathlib import Path
from typing import List
def find_result_folders(base_path: Path, pattern_name: str) -> List[Path]:
    return [p for p in base_path.glob(pattern_name) if p.is_dir()]

def zip_folder(folder_path: Path, output_base_path: Path) -> bool:
    folder_name = folder_path.name
    zip_path = output_base_path / f"{folder_name}.zip"
    try:
        print(f"   -> Zipping folder: {folder_name}...")
        with zipfile.ZipFile(zip_path, mode="w", compression=zipfile.ZIP_DEFLATED) as zipf:
            for file_path in folder_path.rglob("*"):
                if file_path.is_file():
                    arcname = file_path.relative_to(folder_path.parent)
                    zipf.write(file_path, arcname)
        print(f"   ✅ Created ZIP: {zip_path.name}")
        return True
    except Exception as exc:
        print(f"   ❌ Failed to zip {folder_name}: {exc}")
        return False

def zip_stats_results_folders(output_base_path: str, pattern_name: str) -> None:
    base = Path(output_base_path)
    base.mkdir(parents=True, exist_ok=True)
    result_folders = find_result_folders(base, pattern_name)
    if not result_folders:
        print(f"⚠️ No folders matching '*dataset' found in '{output_base_path}'.")
        return
    print(f"🔍 Found {len(result_folders)} result folder(s) to zip.")
    successful = sum(1 for folder in result_folders if zip_folder(folder, base))
    print(f"\n✅ DONE! Successfully zipped {successful} out of {len(result_folders)} folder(s).")

if __name__ == "__main__":
    try:
        output_root = os.getenv("OUTPUT_PATH") or globals().get("OUTPUT_PATH")
        if not output_root:
            raise ValueError("OUTPUT_PATH not defined")
        zip_stats_results_folders(
            output_base_path=OUTPUT_PATH,
            pattern_name="my_dataset")
    except Exception as e:
        print(f"❌ An error occurred: {e}")